In [2]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('xbt_1min_31-10_05-11.csv')

In [20]:
df5 = pd.read_csv('xbt_5min_08-10_05-11.csv')

In [12]:
def pre_process(data):
    
    temp = data.copy()
    
    #Making lowercase all the column names
    temp.columns = map(str.lower, temp.columns)
    
    #Changing the name of the Time column
    try:
        temp.rename(columns={'time':'timestamp'}, inplace=True)
    except:
        pass
    
    #Converting Unix Timestamp to Datetime format
    if('timestamp' in temp.columns):
        temp['timestamp'] = pd.to_datetime(temp['timestamp'])

    #Making Time column as the index
    if('timestamp' in temp.columns):
        temp.set_index('timestamp', inplace=True)
    
    return temp

In [49]:
def data_rollup(data,candlestcik_length):
    
    temp = data.copy()
    candlestcik_length = '{}'.format(candlestcik_length) +'min'
    
    temp['open'] = temp['open'].resample(candlestcik_length, label='right', closed = 'right').ohlc()['open']
    temp['high'] = temp['high'].resample(candlestcik_length, label='right', closed = 'right').ohlc()['high']
    temp['low'] = temp['low'].resample(candlestcik_length, label='right', closed = 'right').ohlc()['low']
    temp['close'] = temp['close'].resample(candlestcik_length, label='right', closed = 'right').ohlc()['close']
    temp['volume'] = temp['volume'].resample(candlestcik_length, label='right', closed = 'right').sum()
    
    #Drop na values based on any one na among OHLCV. As a matter of fact, all will be na.
    temp.dropna(how='any',inplace=True)
    
    return temp

In [58]:
pdf = pre_process(df)

In [101]:
# Correct implementation. Using for loop.
def rsi(data, rsi_win):

    def gain(x):
        if(pd.isnull(x)):
            return np.nan
        elif(x>0):
            return x
        else:
            return 0
    
    def loss(x):
        if(pd.isnull(x)):
            return np.nan
        elif(x<0):
            return abs(x)
        else:
            return 0
    
    temp = data.copy()
    
    temp['change'] = temp['close'] - temp['close'].shift(1)
    temp['gain'] = temp['change'].apply(lambda x: gain(x)) 
    temp['loss'] = temp['change'].apply(lambda x: loss(x)) 
    
    initial_cols = temp.columns
    
    for win in rsi_win:
        
        temp['avg_gain'] = np.nan
        temp['avg_loss'] = np.nan

        temp.iloc[win, temp.columns.get_loc('avg_gain')] = temp['gain'].rolling(win).mean().iloc[win]
        temp.iloc[win, temp.columns.get_loc('avg_loss')] = temp['loss'].rolling(win).mean().iloc[win]
        
        for i in range(win+1,temp.shape[0]):
            print(win, i)
            temp.iloc[i, temp.columns.get_loc('avg_gain')] = ((temp['avg_gain'].iloc[i-1] * (win-1)) + temp['gain'].iloc[i]) / win  
            temp.iloc[i, temp.columns.get_loc('avg_loss')] = ((temp['avg_loss'].iloc[i-1] * (win-1)) + temp['loss'].iloc[i]) / win           
            

        temp['rs'] = temp['avg_gain'] / temp['avg_loss']
        temp['rsi'+str(win)] = 100 - (100/(1+temp['rs']))
        temp.drop('rs', axis = 1, inplace = True)
    
#     temp.drop(initial_cols, axis = 1, inplace = True)
#     temp.drop(['avg_gain','avg_loss'],axis=1,inplace=True)
    
    return temp

In [148]:
# Correct implementation. Using for loop.
def rsi1(data, rsi_win):

    def gain(x):
        if(pd.isnull(x)):
            return np.nan
        elif(x>0):
            return x
        else:
            return 0
    
    def loss(x):
        if(pd.isnull(x)):
            return np.nan
        elif(x<0):
            return abs(x)
        else:
            return 0
    
    temp = data.copy()
    
    temp['change'] = temp['close'] - temp['close'].shift(1)
    temp['gain'] = temp['change'].apply(lambda x: gain(x)) 
    temp['loss'] = temp['change'].apply(lambda x: loss(x)) 
    
    initial_cols = temp.columns
    
    for win in rsi_win:
        
        temp['avg_gain'] = temp['gain'].ewm(com=win - 1,  adjust=True).mean()
        temp['avg_loss'] = temp['loss'].ewm(com=win - 1,  adjust=True).mean()

#         temp.iloc[win, temp.columns.get_loc('avg_gain')] = temp['gain'].rolling(win).mean().iloc[win]
#         temp.iloc[win, temp.columns.get_loc('avg_loss')] = temp['loss'].rolling(win).mean().iloc[win]
        
#         for i in range(win+1,temp.shape[0]):
#             print(win, i)
#             temp.iloc[i, temp.columns.get_loc('avg_gain')] = ((temp['avg_gain'].iloc[i-1] * (win-1)) + temp['gain'].iloc[i]) / win  
#             temp.iloc[i, temp.columns.get_loc('avg_loss')] = ((temp['avg_loss'].iloc[i-1] * (win-1)) + temp['loss'].iloc[i]) / win           
            

        temp['rs'] = temp['avg_gain'] / temp['avg_loss']
        temp['rsi'+str(win)] = 100 - (100/(1+temp['rs']))
        temp.drop('rs', axis = 1, inplace = True)
    
#     temp.drop(initial_cols, axis = 1, inplace = True)
#     temp.drop(['avg_gain','avg_loss'],axis=1,inplace=True)
    
    return temp

In [61]:
rsi_data.iloc[14,rsi_data.columns.get_loc('rsi14')]

7.692307692307679

In [156]:
rsi_data = rsi(data_rollup(pre_process(df),1),[14])

14 15
14 16
14 17
14 18
14 19
14 20
14 21
14 22
14 23
14 24
14 25
14 26
14 27
14 28
14 29
14 30
14 31
14 32
14 33
14 34
14 35
14 36
14 37
14 38
14 39
14 40
14 41
14 42
14 43
14 44
14 45
14 46
14 47
14 48
14 49
14 50
14 51
14 52
14 53
14 54
14 55
14 56
14 57
14 58
14 59
14 60
14 61
14 62
14 63
14 64
14 65
14 66
14 67
14 68
14 69
14 70
14 71
14 72
14 73
14 74
14 75
14 76
14 77
14 78
14 79
14 80
14 81
14 82
14 83
14 84
14 85
14 86
14 87
14 88
14 89
14 90
14 91
14 92
14 93
14 94
14 95
14 96
14 97
14 98
14 99
14 100
14 101
14 102
14 103
14 104
14 105
14 106
14 107
14 108
14 109
14 110
14 111
14 112
14 113
14 114
14 115
14 116
14 117
14 118
14 119
14 120
14 121
14 122
14 123
14 124
14 125
14 126
14 127
14 128
14 129
14 130
14 131
14 132
14 133
14 134
14 135
14 136
14 137
14 138
14 139
14 140
14 141
14 142
14 143
14 144
14 145
14 146
14 147
14 148
14 149
14 150
14 151
14 152
14 153
14 154
14 155
14 156
14 157
14 158
14 159
14 160
14 161
14 162
14 163
14 164
14 165
14 166
14 167
14 168
14 169


14 1240
14 1241
14 1242
14 1243
14 1244
14 1245
14 1246
14 1247
14 1248
14 1249
14 1250
14 1251
14 1252
14 1253
14 1254
14 1255
14 1256
14 1257
14 1258
14 1259
14 1260
14 1261
14 1262
14 1263
14 1264
14 1265
14 1266
14 1267
14 1268
14 1269
14 1270
14 1271
14 1272
14 1273
14 1274
14 1275
14 1276
14 1277
14 1278
14 1279
14 1280
14 1281
14 1282
14 1283
14 1284
14 1285
14 1286
14 1287
14 1288
14 1289
14 1290
14 1291
14 1292
14 1293
14 1294
14 1295
14 1296
14 1297
14 1298
14 1299
14 1300
14 1301
14 1302
14 1303
14 1304
14 1305
14 1306
14 1307
14 1308
14 1309
14 1310
14 1311
14 1312
14 1313
14 1314
14 1315
14 1316
14 1317
14 1318
14 1319
14 1320
14 1321
14 1322
14 1323
14 1324
14 1325
14 1326
14 1327
14 1328
14 1329
14 1330
14 1331
14 1332
14 1333
14 1334
14 1335
14 1336
14 1337
14 1338
14 1339
14 1340
14 1341
14 1342
14 1343
14 1344
14 1345
14 1346
14 1347
14 1348
14 1349
14 1350
14 1351
14 1352
14 1353
14 1354
14 1355
14 1356
14 1357
14 1358
14 1359
14 1360
14 1361
14 1362
14 1363
14 1364


14 2323
14 2324
14 2325
14 2326
14 2327
14 2328
14 2329
14 2330
14 2331
14 2332
14 2333
14 2334
14 2335
14 2336
14 2337
14 2338
14 2339
14 2340
14 2341
14 2342
14 2343
14 2344
14 2345
14 2346
14 2347
14 2348
14 2349
14 2350
14 2351
14 2352
14 2353
14 2354
14 2355
14 2356
14 2357
14 2358
14 2359
14 2360
14 2361
14 2362
14 2363
14 2364
14 2365
14 2366
14 2367
14 2368
14 2369
14 2370
14 2371
14 2372
14 2373
14 2374
14 2375
14 2376
14 2377
14 2378
14 2379
14 2380
14 2381
14 2382
14 2383
14 2384
14 2385
14 2386
14 2387
14 2388
14 2389
14 2390
14 2391
14 2392
14 2393
14 2394
14 2395
14 2396
14 2397
14 2398
14 2399
14 2400
14 2401
14 2402
14 2403
14 2404
14 2405
14 2406
14 2407
14 2408
14 2409
14 2410
14 2411
14 2412
14 2413
14 2414
14 2415
14 2416
14 2417
14 2418
14 2419
14 2420
14 2421
14 2422
14 2423
14 2424
14 2425
14 2426
14 2427
14 2428
14 2429
14 2430
14 2431
14 2432
14 2433
14 2434
14 2435
14 2436
14 2437
14 2438
14 2439
14 2440
14 2441
14 2442
14 2443
14 2444
14 2445
14 2446
14 2447


14 3441
14 3442
14 3443
14 3444
14 3445
14 3446
14 3447
14 3448
14 3449
14 3450
14 3451
14 3452
14 3453
14 3454
14 3455
14 3456
14 3457
14 3458
14 3459
14 3460
14 3461
14 3462
14 3463
14 3464
14 3465
14 3466
14 3467
14 3468
14 3469
14 3470
14 3471
14 3472
14 3473
14 3474
14 3475
14 3476
14 3477
14 3478
14 3479
14 3480
14 3481
14 3482
14 3483
14 3484
14 3485
14 3486
14 3487
14 3488
14 3489
14 3490
14 3491
14 3492
14 3493
14 3494
14 3495
14 3496
14 3497
14 3498
14 3499
14 3500
14 3501
14 3502
14 3503
14 3504
14 3505
14 3506
14 3507
14 3508
14 3509
14 3510
14 3511
14 3512
14 3513
14 3514
14 3515
14 3516
14 3517
14 3518
14 3519
14 3520
14 3521
14 3522
14 3523
14 3524
14 3525
14 3526
14 3527
14 3528
14 3529
14 3530
14 3531
14 3532
14 3533
14 3534
14 3535
14 3536
14 3537
14 3538
14 3539
14 3540
14 3541
14 3542
14 3543
14 3544
14 3545
14 3546
14 3547
14 3548
14 3549
14 3550
14 3551
14 3552
14 3553
14 3554
14 3555
14 3556
14 3557
14 3558
14 3559
14 3560
14 3561
14 3562
14 3563
14 3564
14 3565


14 4543
14 4544
14 4545
14 4546
14 4547
14 4548
14 4549
14 4550
14 4551
14 4552
14 4553
14 4554
14 4555
14 4556
14 4557
14 4558
14 4559
14 4560
14 4561
14 4562
14 4563
14 4564
14 4565
14 4566
14 4567
14 4568
14 4569
14 4570
14 4571
14 4572
14 4573
14 4574
14 4575
14 4576
14 4577
14 4578
14 4579
14 4580
14 4581
14 4582
14 4583
14 4584
14 4585
14 4586
14 4587
14 4588
14 4589
14 4590
14 4591
14 4592
14 4593
14 4594
14 4595
14 4596
14 4597
14 4598
14 4599
14 4600
14 4601
14 4602
14 4603
14 4604
14 4605
14 4606
14 4607
14 4608
14 4609
14 4610
14 4611
14 4612
14 4613
14 4614
14 4615
14 4616
14 4617
14 4618
14 4619
14 4620
14 4621
14 4622
14 4623
14 4624
14 4625
14 4626
14 4627
14 4628
14 4629
14 4630
14 4631
14 4632
14 4633
14 4634
14 4635
14 4636
14 4637
14 4638
14 4639
14 4640
14 4641
14 4642
14 4643
14 4644
14 4645
14 4646
14 4647
14 4648
14 4649
14 4650
14 4651
14 4652
14 4653
14 4654
14 4655
14 4656
14 4657
14 4658
14 4659
14 4660
14 4661
14 4662
14 4663
14 4664
14 4665
14 4666
14 4667


14 5658
14 5659
14 5660
14 5661
14 5662
14 5663
14 5664
14 5665
14 5666
14 5667
14 5668
14 5669
14 5670
14 5671
14 5672
14 5673
14 5674
14 5675
14 5676
14 5677
14 5678
14 5679
14 5680
14 5681
14 5682
14 5683
14 5684
14 5685
14 5686
14 5687
14 5688
14 5689
14 5690
14 5691
14 5692
14 5693
14 5694
14 5695
14 5696
14 5697
14 5698
14 5699
14 5700
14 5701
14 5702
14 5703
14 5704
14 5705
14 5706
14 5707
14 5708
14 5709
14 5710
14 5711
14 5712
14 5713
14 5714
14 5715
14 5716
14 5717
14 5718
14 5719
14 5720
14 5721
14 5722
14 5723
14 5724
14 5725
14 5726
14 5727
14 5728
14 5729
14 5730
14 5731
14 5732
14 5733
14 5734
14 5735
14 5736
14 5737
14 5738
14 5739
14 5740
14 5741
14 5742
14 5743
14 5744
14 5745
14 5746
14 5747
14 5748
14 5749
14 5750
14 5751
14 5752
14 5753
14 5754
14 5755
14 5756
14 5757
14 5758
14 5759
14 5760
14 5761
14 5762
14 5763
14 5764
14 5765
14 5766
14 5767
14 5768
14 5769
14 5770
14 5771
14 5772
14 5773
14 5774
14 5775
14 5776
14 5777
14 5778
14 5779
14 5780
14 5781
14 5782


14 6739
14 6740
14 6741
14 6742
14 6743
14 6744
14 6745
14 6746
14 6747
14 6748
14 6749
14 6750
14 6751
14 6752
14 6753
14 6754
14 6755
14 6756
14 6757
14 6758
14 6759
14 6760
14 6761
14 6762
14 6763
14 6764
14 6765
14 6766
14 6767
14 6768
14 6769
14 6770
14 6771
14 6772
14 6773
14 6774
14 6775
14 6776
14 6777
14 6778
14 6779
14 6780
14 6781
14 6782
14 6783
14 6784
14 6785
14 6786
14 6787
14 6788
14 6789
14 6790
14 6791
14 6792
14 6793
14 6794
14 6795
14 6796
14 6797
14 6798
14 6799
14 6800
14 6801
14 6802
14 6803
14 6804
14 6805
14 6806
14 6807
14 6808
14 6809
14 6810
14 6811
14 6812
14 6813
14 6814
14 6815
14 6816
14 6817
14 6818
14 6819
14 6820
14 6821
14 6822
14 6823
14 6824
14 6825
14 6826
14 6827
14 6828
14 6829
14 6830
14 6831
14 6832
14 6833
14 6834
14 6835
14 6836
14 6837
14 6838
14 6839
14 6840
14 6841
14 6842
14 6843
14 6844
14 6845
14 6846
14 6847
14 6848
14 6849
14 6850
14 6851
14 6852
14 6853
14 6854
14 6855
14 6856
14 6857
14 6858
14 6859
14 6860
14 6861
14 6862
14 6863


14 7785
14 7786
14 7787
14 7788
14 7789
14 7790
14 7791
14 7792
14 7793
14 7794
14 7795
14 7796
14 7797
14 7798
14 7799
14 7800
14 7801
14 7802
14 7803
14 7804
14 7805
14 7806
14 7807
14 7808
14 7809
14 7810
14 7811
14 7812
14 7813
14 7814
14 7815
14 7816
14 7817
14 7818
14 7819
14 7820
14 7821
14 7822
14 7823
14 7824
14 7825
14 7826
14 7827
14 7828
14 7829
14 7830
14 7831
14 7832
14 7833
14 7834
14 7835
14 7836
14 7837
14 7838
14 7839
14 7840
14 7841
14 7842
14 7843
14 7844
14 7845
14 7846
14 7847
14 7848
14 7849
14 7850
14 7851
14 7852
14 7853
14 7854
14 7855
14 7856
14 7857
14 7858
14 7859
14 7860
14 7861
14 7862
14 7863
14 7864
14 7865
14 7866
14 7867
14 7868
14 7869
14 7870
14 7871
14 7872
14 7873
14 7874
14 7875
14 7876
14 7877
14 7878
14 7879
14 7880
14 7881
14 7882
14 7883
14 7884
14 7885
14 7886
14 7887
14 7888
14 7889
14 7890
14 7891
14 7892
14 7893
14 7894
14 7895
14 7896
14 7897
14 7898
14 7899
14 7900
14 7901
14 7902
14 7903
14 7904
14 7905
14 7906
14 7907
14 7908
14 7909


In [157]:
rsi_data

,open,high,low,close,volume,change,gain,loss,avg_gain,avg_loss,rsi14
timestamp,,,,,,,,,,,
2018-10-31 00:31:00,6278.5,6279.0,6278.5,6278.5,85286,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-31 00:32:00,6278.5,6279.0,6278.5,6278.5,153512,0.0,0.0,0.0,NaN,NaN,NaN
2018-10-31 00:33:00,6278.5,6279.0,6278.5,6279.0,15763,0.5,0.5,0.0,NaN,NaN,NaN
2018-10-31 00:34:00,6279.0,6279.0,6278.5,6278.5,365214,-0.5,0.0,0.5,NaN,NaN,NaN
2018-10-31 00:35:00,6278.5,6279.0,6278.5,6279.0,153937,0.5,0.5,0.0,NaN,NaN,NaN
2018-10-31 00:36:00,6279.0,6279.0,6278.5,6278.5,1049550,-0.5,0.0,0.5,NaN,NaN,NaN
2018-10-31 00:37:00,6278.5,6279.0,6278.5,6279.0,174762,0.5,0.5,0.0,NaN,NaN,NaN
2018-10-31 00:38:00,6279.0,6279.0,6278.5,6279.0,144456,0.0,0.0,0.0,NaN,NaN,NaN
2018-10-31 00:39:00,6279.0,6279.0,6279.0,6279.0,68827,0.0,0.0,0.0,NaN,NaN,NaN


In [158]:
rsi_data1 = rsi1(data_rollup(pre_process(df),1),[14])

In [135]:
rsi_data2 = rsi1(data_rollup(pre_process(df),1),[7])

In [146]:
rsi_data2['rsi7'] == rsi_data1['rsi7']

timestamp
2018-10-31 00:31:00    False
2018-10-31 00:32:00    False
2018-10-31 00:33:00     True
2018-10-31 00:34:00     True
2018-10-31 00:35:00     True
2018-10-31 00:36:00     True
2018-10-31 00:37:00     True
2018-10-31 00:38:00     True
2018-10-31 00:39:00     True
2018-10-31 00:40:00     True
2018-10-31 00:41:00     True
2018-10-31 00:42:00     True
2018-10-31 00:43:00     True
2018-10-31 00:44:00     True
2018-10-31 00:45:00     True
2018-10-31 00:46:00     True
2018-10-31 00:47:00     True
2018-10-31 00:48:00     True
2018-10-31 00:49:00     True
2018-10-31 00:50:00     True
2018-10-31 00:51:00     True
2018-10-31 00:52:00     True
2018-10-31 00:53:00     True
2018-10-31 00:54:00     True
2018-10-31 00:55:00     True
2018-10-31 00:56:00     True
2018-10-31 00:57:00     True
2018-10-31 00:58:00     True
2018-10-31 00:59:00     True
2018-10-31 01:00:00     True
                       ...  
2018-11-05 17:31:00     True
2018-11-05 17:32:00     True
2018-11-05 17:33:00     True
2018

In [153]:
pd.set_option('display.max_rows', 5000)

In [159]:
rsi_data['rsi14'] - rsi_data1['rsi14']

timestamp
2018-10-31 00:31:00             NaN
2018-10-31 00:32:00             NaN
2018-10-31 00:33:00             NaN
2018-10-31 00:34:00             NaN
2018-10-31 00:35:00             NaN
2018-10-31 00:36:00             NaN
2018-10-31 00:37:00             NaN
2018-10-31 00:38:00             NaN
2018-10-31 00:39:00             NaN
2018-10-31 00:40:00             NaN
2018-10-31 00:41:00             NaN
2018-10-31 00:42:00             NaN
2018-10-31 00:43:00             NaN
2018-10-31 00:44:00             NaN
2018-10-31 00:45:00    4.638866e+00
2018-10-31 00:46:00    7.920422e-01
2018-10-31 00:47:00    7.920422e-01
2018-10-31 00:48:00    7.920422e-01
2018-10-31 00:49:00    7.920422e-01
2018-10-31 00:50:00    7.920422e-01
2018-10-31 00:51:00    7.920422e-01
2018-10-31 00:52:00    7.920422e-01
2018-10-31 00:53:00    7.920422e-01
2018-10-31 00:54:00    4.395072e+00
2018-10-31 00:55:00    4.395072e+00
2018-10-31 00:56:00    4.395072e+00
2018-10-31 00:57:00    5.816383e+00
2018-10-31 00:58:0

In [133]:
rsi_data1

,open,high,low,close,volume,change,gain,loss,avg_gain,avg_loss,rsi7
timestamp,,,,,,,,,,,
2018-10-31 00:31:00,6278.5,6279.0,6278.5,6278.5,85286,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-31 00:32:00,6278.5,6279.0,6278.5,6278.5,153512,0.0,0.0,0.0,0.000000,0.000000,NaN
2018-10-31 00:33:00,6278.5,6279.0,6278.5,6279.0,15763,0.5,0.5,0.0,0.269231,0.000000,100.000000
2018-10-31 00:34:00,6279.0,6279.0,6278.5,6278.5,365214,-0.5,0.0,0.5,0.165354,0.192913,46.153846
2018-10-31 00:35:00,6278.5,6279.0,6278.5,6279.0,153937,0.5,0.5,0.0,0.269231,0.133032,66.929134
2018-10-31 00:36:00,6279.0,6279.0,6278.5,6278.5,1049550,-0.5,0.0,0.5,0.197653,0.230595,46.153846
2018-10-31 00:37:00,6278.5,6279.0,6278.5,6279.0,174762,0.5,0.5,0.0,0.269231,0.176003,60.469494
2018-10-31 00:38:00,6279.0,6279.0,6278.5,6279.0,144456,0.0,0.0,0.0,0.210963,0.137912,60.469494
2018-10-31 00:39:00,6279.0,6279.0,6279.0,6279.0,68827,0.0,0.0,0.0,0.168434,0.110110,60.469494
